In [1]:
import pandas as pd
import datetime as dtime

df = pd.read_csv('SMSHistory.csv')
df.head()

,ID,User Creation Date,Date Created,Date Updated,Date Sent,User,From,To,Status,Text,NewPh,Unnamed: 11
0,SM003264a252114,6/25/2019 20:57,6/25/2019 18:00,6/25/2019 18:04,6/25/2019 18:04,Randy,+1 123-456-7890,+1 123-456-7890,undelivered,Switch to...,NaN,NaN
1,SM02ca93a427164,6/25/2019 20:57,6/25/2019 17:59,6/25/2019 17:59,6/25/2019 17:59,Randy,+1 123-456-7890,+1 123-456-7890,undelivered,Switch to...,NaN,NaN


In [2]:
db_sent = df[df['To'].notnull()]

print('Sent DB:', '\n',
      'Full base     -> ', len(df.index), '\n',
      'Without nulls -> ', len(db_sent.index))

# db_sent['NewPh'] = ''
db_sent          = db_sent.astype({"NewPh": str, "To": str})
db_sent          = db_sent[['To', 'NewPh']]
db_sent['Code']  = db_sent['To'].str[:2]
db_sent          = db_sent.query('Code == "+1"')

print(' Only US       -> ', len(db_sent.index))

db_sent.head()

# for index, row in db_sent.iterrows():
#     number     = db_sent.loc[index, 'To']
#     new_number = number[3:]
#     db_sent.at[index, 'NewPh'] = str(new_number)

db_sent['NewPh'] = db_sent['To'].str[3:]

print(' Double check...  ', len(db_sent.index))
db_sent.head()

Sent DB: 
 Full base     ->  91252 
 Without nulls ->  91101
 Only US       ->  90187
 Double check...   90187


,To,NewPh,Code
0,+1 123-456-7890,123-456-7890,+1
1,+1 123-456-7890,123-456-7890,+1


In [3]:
db_Main     = pd.read_csv('IN4Main.csv')
full_db_ln  = len(db_Main.index)

db_Main     = db_Main[db_Main['PHONE'].notnull()]
db_Main     = db_Main.astype({"NewPh": str, "PHONE": str})
not_null_ln = len(db_Main.index)

print('Main DB:', '\n',
      'Full base     -> ', full_db_ln, '\n',
      'Without nulls -> ', not_null_ln)

db_Main['NewPh'] = db_Main['PHONE'].str[1:4] + '-' + db_Main['PHONE'].str[6:]

# f = 0
# for index, row in db_Main.iterrows():
# #     if (row['PHONE']) == 'nan': #python paradox... :0
# #         print((row['PHONE']))
# #         f += 1
#     number     = db_Main.loc[index, 'PHONE']
#     new_number = number[1:]
#     db_Main.at[index, 'NewPh'] = str(new_number[:3] + '-' + str(new_number[5:]))
#     print(type(row['To']), type(row['NewPh']))
# print(f)

db_Main.head()

Main DB: 
 Full base     ->  19808 
 Without nulls ->  19642


,RRID,SEARCH_COUNT,USDOT,ENT_COMPANY1,ENT_COMPANY2,1st Contact,2st Contact,ENT_CONTACT2_FIRST,ENT_CONTACT2_LAST,PHONE,...,ACC_FATALITY_TOTAL,ACC_INJURY_TOTAL,RATING,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,NewPh
0,01016434,9,0101087,1St CDS,NaN,Zachely,Porson,NaN,NaN,(123) 456-789,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123-456-7890
1,010195,9,0101556,BA LLC,BJ Trucking,Bob,Louderback,NaN,NaN,(123) 456-7890,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123-456-7890


In [4]:
list_sent = db_sent['NewPh'].tolist()
list_Main = db_Main['NewPh'].tolist()
print(' Total numbers in sent DB -> ', len(list_sent), '\n',
      'Total numbers in main DB -> ', len(list_Main), '\n')

s_sent    = set(list_sent)
s_Main    = set(list_Main)

s_result  = s_Main.difference(s_sent)
print(' Unique num. in sent DB   -> ', len(s_sent), '\n',
      'Unique num. in main DB   -> ', len(s_Main), '\n\n',
      'Unique num. in result DB -> ', len(s_result))

 Total numbers in sent DB ->  90187 
 Total numbers in main DB ->  19642 

 Unique num. in sent DB   ->  75254 
 Unique num. in main DB   ->  19001 

 Unique num. in result DB ->  16206


In [5]:
cq = int(input('Quantity per file ')) + 1
ci = 0
i  = 1

def create_ini():
    timenow = dtime.datetime.now()
    name    = timenow.strftime("%Y-%m-%d_%H-%M-%f")
    print(name)
    f_ini   = open(f'export_{name}.ini', 'w')
    f_ini.write('[contacts]\n')
    f_ini.write('size=\n')
    return f_ini

def save_close(f_ini):
    #f_ini
    f_ini.close()
    print('+ f')

for customer in s_result:
    if ci == cq:
        save_close(f_ini)
        f_ini = create_ini()
        ci    = 0
        i     = 1
    elif ci == 0:
        f_ini = create_ini()
    else:
        qr     = db_Main.query('NewPh == @customer')
        ph_num = qr['PHONE'].values[0]
        #print(ph_num, ' ', len(ph_num))
        if len(ph_num) == 14:
            c_name = str(qr['1st Contact'].values[0]) + ' ' + str(qr['2st Contact'].values[0])
            f_ini.write(f'{i}\\Phone={ph_num}\n')
            f_ini.write(f'{i}\\Name={c_name}\n')
            i += 1
    ci += 1
save_close(f_ini)
print('Done!')

Quantity per file 500
2019-06-28_08-22-768375
+ f
2019-06-28_08-22-897440
+ f
2019-06-28_08-22-028505
+ f
2019-06-28_08-22-172570
+ f
2019-06-28_08-22-313635
+ f
2019-06-28_08-22-443700
+ f
2019-06-28_08-22-623768
+ f
2019-06-28_08-22-846837
+ f
2019-06-28_08-23-075908
+ f
2019-06-28_08-23-332980
+ f
2019-06-28_08-23-589052
+ f
2019-06-28_08-23-728117
+ f
2019-06-28_08-23-930185
+ f
2019-06-28_08-23-118253
+ f
2019-06-28_08-23-296321
+ f
2019-06-28_08-23-477388
+ f
2019-06-28_08-23-657456
+ f
2019-06-28_08-23-859525
+ f
2019-06-28_08-23-021591
+ f
2019-06-28_08-23-196658
+ f
2019-06-28_08-23-343724
+ f
2019-06-28_08-23-482789
+ f
2019-06-28_08-23-637855
+ f
2019-06-28_08-23-784921
+ f
2019-06-28_08-23-953988
+ f
2019-06-28_08-23-086052
+ f
2019-06-28_08-23-250119
+ f
2019-06-28_08-23-362182
+ f
2019-06-28_08-23-451245
+ f
2019-06-28_08-23-607311
+ f
2019-06-28_08-23-871383
+ f
2019-06-28_08-23-995447
+ f
2019-06-28_08-23-158514
+ f
Done!


START for Nick's case:

In [ ]:
import pandas as pd
import datetime as dtime

db_Main = pd.read_csv('KolyaEng.csv')
print(len(db_Main.index))

db_Main['NewPh'] = ''
db_Main = db_Main[db_Main['PHONE'].notnull()]
db_Main = db_Main.astype({"NewPh": str, "PHONE": str})
print(len(db_Main.index))

db_Main['NewPh'] = db_Main['PHONE'].str[1:4] + '-' + db_Main['PHONE'].str[6:]

db_Main.head()

In [ ]:
list_Main = db_Main['NewPh'].tolist()
print(len(list_Main))

s_Main    = set(list_Main)

print(len(s_Main))

In [ ]:
cq = int(input('Quantity per file ')) + 1
ci = 0
i  = 1

def create_ini():
    timenow = dtime.datetime.now()
    name    = timenow.strftime("%Y-%m-%d_%H-%M-%f")
    print(name)
    f_ini   = open(f'export_{name}.ini', 'w')
    f_ini.write('[contacts]\n')
    f_ini.write('size=\n')
    return f_ini

def save_close(f_ini):
    f_ini.close()
    print('+ f')

for customer in s_Main:
    if ci == cq:
        save_close(f_ini)
        f_ini = create_ini()
        ci    = 0
        i     = 1
    elif ci == 0:
        f_ini = create_ini()
    else:
        qr     = db_Main.query('NewPh == @customer')
        ph_num = qr['PHONE'].values[0]
        #print(ph_num, ' ', len(ph_num))
        if len(ph_num) == 14:
            c_name = str(qr['NAME'].values[0])
            f_ini.write(f'{i}\\Phone={ph_num}\n')
            f_ini.write(f'{i}\\Name={c_name}\n')
            i += 1
    ci += 1
save_close(f_ini)
print('Done!')

END for Nick's case:

START SMS log case:

In [ ]:
import pandas as pd
import datetime as dtime

db_Main = pd.read_csv('sms-log-2019-04-30.csv')
print(len(db_Main.index))

db_Main.head()
df = db_Main[['From', 'To', 'Body']]
df['txt'] = df['Body'].str[:6]
df = df.query('txt != "Switch" and txt != "No" and txt != "Yes" and txt != "STOP" and txt != "Stop " and txt != "Stop"')
for i, row in df.iterrows():
    print(row['From'], '==' , row['Body'])

END SMS log case: